In [2]:
# Part-1: Data Cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

rounds=pd.read_csv("rounds_cleaned.csv",encoding="ISO-8859-1")
companies=pd.read_csv('companies_cleaned.csv',sep="\t",encoding="ISO-8859-1")

In [3]:
print(len(companies.permalink.unique()))
print(len(rounds.company_permalink.unique()))

66368
66368


In [4]:
#Missing Value Treatment...

In [6]:
companies.isnull().sum()

permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6958
state_code        8547
region            8030
city              8028
founded_at       15221
dtype: int64

In [7]:
rounds.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
dtype: int64

In [8]:
#Common  column between the 2 files ----> permalink

In [9]:
merged=pd.merge(companies,rounds,how='inner',left_on='permalink',right_on='company_permalink')

In [10]:
merged.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [12]:
merged.drop('company_permalink',1,inplace=True)

In [13]:
merged.isnull().sum()

permalink                      0
name                           1
homepage_url                6126
category_list               3387
status                         0
country_code                8637
state_code                 10903
region                     10126
city                       10123
founded_at                 20488
funding_round_permalink        0
funding_round_type             0
funding_round_code         83743
funded_at                      0
raised_amount_usd          19960
dtype: int64

In [14]:
#Missing values percentage
round(100*(merged.isnull().sum()/len(merged.index)),2)

permalink                   0.00
name                        0.00
homepage_url                5.33
category_list               2.95
status                      0.00
country_code                7.52
state_code                  9.49
region                      8.81
city                        8.81
founded_at                 17.84
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.90
funded_at                   0.00
raised_amount_usd          17.38
dtype: float64

In [15]:
# funding_round_code with  72.90 % missing values is  useless, homepage_url is also useless
merged.drop(['funding_round_code','homepage_url','founded_at','state_code','region','city'],axis=1,inplace=True)

In [18]:
round(100*(merged.isnull().sum()/len(merged.index)),2)

permalink                   0.00
name                        0.00
category_list               2.95
status                      0.00
country_code                7.52
funding_round_permalink     0.00
funding_round_type          0.00
funded_at                   0.00
raised_amount_usd          17.38
dtype: float64

In [20]:
merged['raised_amount_usd'].describe()

count    9.491500e+04
mean     1.043057e+07
std      1.148477e+08
min      0.000000e+00
25%      3.241400e+05
50%      1.686193e+06
75%      7.000000e+06
max      2.127194e+10
Name: raised_amount_usd, dtype: float64

In [21]:
#Removing Nans in raised_amount_usd
merged=merged[~np.isnan(merged['raised_amount_usd'])]
round(100*(merged.isnull().sum()/len(merged.index)),2)

permalink                  0.00
name                       0.00
category_list              1.09
status                     0.00
country_code               6.14
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
dtype: float64

In [22]:
country_codes=merged['country_code'].astype('category')
country_codes.value_counts()

USA    62045
GBR     5018
CAN     2616
CHN     1927
IND     1649
       ...  
HND        1
GRD        1
GGY        1
DZA        1
KNA        1
Name: country_code, Length: 134, dtype: int64

In [23]:
#fraction of country counts
round(100*(merged['country_code'].value_counts()/len(merged.index)),2)

USA    65.37
GBR     5.29
CAN     2.76
CHN     2.03
IND     1.74
       ...  
PRY     0.00
UZB     0.00
DZA     0.00
MKD     0.00
OMN     0.00
Name: country_code, Length: 134, dtype: float64

In [27]:
# np.isnan does not work with arrays  of type object, it only works with native numpy type (float). Thus we will use pd.isnull()

In [28]:
#Removing rows with missing country codes...
merged=merged[~pd.isnull(merged['country_code'])]
round(100*(merged.isnull().sum()/len(merged.index)),2)

permalink                  0.00
name                       0.00
category_list              0.65
status                     0.00
country_code               0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
dtype: float64

In [29]:
#Removing rows with missing country codes...
merged=merged[~pd.isnull(merged['category_list'])]
round(100*(merged.isnull().sum()/len(merged.index)),2)

permalink                  0.0
name                       0.0
category_list              0.0
status                     0.0
country_code               0.0
funding_round_permalink    0.0
funding_round_type         0.0
funded_at                  0.0
raised_amount_usd          0.0
dtype: float64

In [30]:
#Cleaned DataFrame
merged.to_csv('cleaned_data.csv',sep=",",index=False)

In [31]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88507 entries, 0 to 114874
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   permalink                88507 non-null  object 
 1   name                     88506 non-null  object 
 2   category_list            88507 non-null  object 
 3   status                   88507 non-null  object 
 4   country_code             88507 non-null  object 
 5   funding_round_permalink  88507 non-null  object 
 6   funding_round_type       88507 non-null  object 
 7   funded_at                88507 non-null  object 
 8   raised_amount_usd        88507 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


In [33]:
#After removing missing values , we were able to rtain 77% of data
100*(len(merged.index)/len(rounds.index))

76.99675508268884